# Imports

In [1]:
from pyiron_contrib.tinybase.task import AbstractTask, FunctionTask, SeriesTask, LoopTask

/home/poul/pyiron/contrib/pyiron_contrib/__init__.py:9: UserWarning: pyiron module not found, importing Project from pyiron_base
  warnings.warn("pyiron module not found, importing Project from pyiron_base")


In [2]:
from pyiron_contrib.tinybase.executor import ProcessExecutor, BackgroundExecutor, Executor

In [3]:
import logging
logging.getLogger().setLevel(20)

In [4]:
import numpy as np

# Function Task

## Basic

In [5]:
def calc_fib(n):
    import time
    n1 = n2 = 1
    for i in range(n):
        time.sleep(.1)
        x = n1 + n2
        n1 = n2
        n2 = x
    return x

In [6]:
f = FunctionTask(calc_fib)

In [7]:
f.input.storage

DataContainer([])

In [8]:
f.input.args

[]

In [9]:
f.input.kwargs

{}

In [10]:
f.input.kwargs['n'] = 10

In [11]:
f.input.kwargs

{'n': 10}

In [12]:
f.execute()

(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fe13b496da0>)

## We can use an executor to distribute the task to any compute resource

### Directly in the foreground

In [13]:
exe = Executor().submit([f])

In [14]:
exe.run()

In [15]:
exe.status[0]

ReturnStatus(Code.DONE, None)

In [16]:
exe.output[0].result

144

### Do the same but in the background

In [17]:
f = FunctionTask(calc_fib)

In [18]:
f.input.kwargs['n'] = 100

In [19]:
exe = BackgroundExecutor(max_threads=1).submit([f])

In [20]:
exe.run()

In [21]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [22]:
exe.wait()

In [23]:
exe.output[0].result

927372692193078999176

### Do the same but in the background as process

In [24]:
f = FunctionTask(calc_fib)

In [25]:
f.input.kwargs['n'] = 100

In [26]:
exe = ProcessExecutor(max_processes=1).submit([f])

In [27]:
exe.run()

In [28]:
exe._run_machine.state

<Code.RUNNING: 'running'>

In [29]:
exe.wait()

In [30]:
exe.output[0].result

927372692193078999176

In [31]:
exe._run_machine.state

<Code.FINISHED: 'finished'>

# Executors handle single Tasks and lists of them on the same footing

In [32]:
tasks = [FunctionTask(calc_fib) for _ in range(10)]

In [33]:
for i, n in enumerate(tasks):
    n.input.kwargs['n'] = 3 + i

## With the basic executor

In [34]:
exe = Executor().submit(tasks)
exe.run()

In [35]:
exe.output

(<pyiron_contrib.tinybase.task.FunctionOutput at 0x7fe13ab04eb0>,
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fe13ab05ba0>)

In [36]:
exe.output[1].result

8

## With the process executor

In [37]:
exe = ProcessExecutor(max_processes=4).submit(tasks)
exe.run()

In [38]:
exe.wait()

In [39]:
exe.status

[ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None),
 ReturnStatus(Code.DONE, None)]

In [40]:
exe.output

In [41]:
exe.output[5].result

55

# SeriesTask

In [42]:
s = SeriesTask()

In [43]:
f1 = FunctionTask(calc_fib)

In [44]:
f2 = FunctionTask(np.sqrt)

In [45]:
def transfer(input, output):
    input.args = [output.result]

In [46]:
s.input.first(f1).then(f2, transfer)

In [47]:
s.input.tasks[0].input.kwargs['n'] = 10

In [48]:
status, output = s.execute()

In [49]:
status

ReturnStatus(Code.DONE, None)

In [50]:
output.result

12.0

# Loop Task

## Simple repeat loop

In [51]:
l = LoopTask()

In [52]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [53]:
l.input.repeat(10, restart=lambda output, input, scratch: print(output.result))

In [54]:
l.execute()

0.951652486500789
0.8479495911177689
0.39209981170147534
0.12626824643855517
0.1151548846424062
0.41394951366874244
0.32349310340372117
0.9985082815924705
0.036743594560547654
0.024840470009968807


(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fe13ab22bf0>)

In [55]:
exe = BackgroundExecutor(max_threads=1).submit([l])
exe.run()
exe.wait()

0.4786223239196473
0.8186548613180863
0.028784009263032373
0.5010370915300685
0.9020777461434385
0.8501369870658283
0.20036590660650433
0.048401602265645605
0.7459809033150049
0.8064230025647129


In [56]:
exe.output[0].result

0.824491968944771

## Loop with a termination condition

In [57]:
l = LoopTask()

In [58]:
l.input.task = FunctionTask(lambda: np.random.rand())

In [59]:
l.input.control_with(
    condition=lambda task, output, scratch: output.result < .15,
    restart=lambda output, input, scratch: print(output.result)
)

In [60]:
l.execute()

0.2292513171117575
0.22555629178899173
0.9524191400881127


(ReturnStatus(Code.DONE, None),
 <pyiron_contrib.tinybase.task.FunctionOutput at 0x7fe13ab3a2f0>)

# Implementation Examples

For a much too simplified example, let's write a task that simply waits `n` times `time` seconds, where each of the `time` waits is a separate, independent task itself.  In tinybase speak such a construct is a `TaskGenerator`, because it internally generates a sequence of atomic tasks that can be scheduled by an executor in whatever order.  From a user's perspective however, a task generator behaves exactly like a task (and it implements the same internal interface).

To write such a class, we need to

1. define an input class;
2. define an output class
3. and combine them on the actual generator.

For the waiting, we'll reuse the already introduces `FunctionTask` to wrap `time.sleep` from the standard library.

In [61]:
from pyiron_contrib.tinybase.task import TaskGenerator, FunctionTask, ReturnStatus
from pyiron_contrib.tinybase.container import AbstractInput, AbstractOutput, StorageAttribute
import time

class WaitInput(AbstractInput):
    # this defines the input parameters
    time = StorageAttribute().type(float).default(10.0)
    n = StorageAttribute().type(int).default(10)

class WaitOutput(AbstractOutput):
    # we have no meaningful output, so we'll leave it empty.
    pass

class WaitGenerator(TaskGenerator):
    # here our task generator class, needs to advertise which input and output classes it is going to use
    def _get_input(self):
        return WaitInput()
    def _get_output(self):
        return WaitOutput()
    def __iter__(self):
        # the main computation in a generator is defined in its __iter__ method.
        # executors will iterate over the the results yielded here and inject back the results
        # in each iteration the generator can dynamically return new tasks depending on the
        # results that came back from an executor.

        # in our case we just have `n` independent waiting tasks, so we create them in a loop
        # and yield them in one iteration; then discard their (anyway empty) outut and return
        # our own return status
        tasks = []
        for _ in range(self.input.n):
            t = FunctionTask(time.sleep)
            tasks.append(t)
            t.input.args = [self.input.time]
        ret, out = zip(*(yield tasks))
        return ReturnStatus.done(), self._get_output()

Passing the `capture_exceptions` as `False` means tinybase will not catch any exceptions
and give us the direct stack trace where any exceptions occured.  This is useful
for debugging a new implemention in a notebook like here.  By default tinybase captures
exceptions and sets the return status to aborted automatically.

In [62]:
%%time
wait = WaitGenerator(capture_exceptions=False)
wait.input.time = 2.0
wait.input.n = 10
wait.execute()

CPU times: user 0 ns, sys: 7.75 ms, total: 7.75 ms
Wall time: 20 s


(ReturnStatus(Code.DONE, None), <__main__.WaitOutput at 0x7fe13ab3b430>)

Calling `execute` on a task generator will simply execute one task after the other.
We therefore expect the run time to be 2 * 10 s.

In [63]:
from pyiron_contrib.tinybase.executor import BackgroundExecutor

If we run with the process executor, but only give one core, we expect the run time
to stay the same.

In [64]:
%%time
exe = ProcessExecutor(max_processes=1).submit([wait])
exe.run()
exe.wait()

CPU times: user 27.8 ms, sys: 32.7 ms, total: 60.5 ms
Wall time: 20.1 s


If we allow multiple cores to wait in parallel the run time naturally goes down accordingly
modulo overhead from the process pool.

In [65]:
%%time
exe = ProcessExecutor(max_processes=4).submit([wait])
exe.run()
exe.wait()

CPU times: user 20.1 ms, sys: 40.7 ms, total: 60.8 ms
Wall time: 6.08 s


Since we are just waiting here, even running in separate threads gives the same speed up, 
regardless of the GIL.

In [66]:
%%time
exe = BackgroundExecutor(max_threads=4).submit([wait])
exe.run()
exe.wait()

CPU times: user 21 ms, sys: 7.34 ms, total: 28.3 ms
Wall time: 6.04 s
